In [64]:
import numpy as np
import pandas as pd
import torch
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
from path import Path

path = Path("../input/modelnet40-princeton-3d-object-dataset/ModelNet40")

In [65]:
def read_off(file):
    off_header = file.readline().strip()
    if 'OFF' == off_header:
        n_verts, n_faces, __ = tuple([int(s) for s in file.readline().strip().split(' ')])
    else:
        n_verts, n_faces, __ = tuple([int(s) for s in off_header[3:].split(' ')])
    verts = [[float(s) for s in file.readline().strip().split(' ')] for i_vert in range(n_verts)]
    return verts

In [66]:
with open(path/"airplane/train/airplane_0001.off", 'r') as f:
    verts = read_off(f)
    
x,y,z = np.array(verts).T
len(x)

90714

Farthest point sampling (FPS) is a technique used to sample a point cloud efficiently and has been used in 3D object detection in algorithms such as Pointnet++ and PV-RCNN. FPS has better coverage over the entire pointset compared to other sampling techniques because it finds a subset of points that are the farthest away from each other. 

In [67]:
def fps(points, n_samples):
    """
    points: [N, 3] array containing the whole point cloud
    n_samples: samples you want in the sampled point cloud typically << N 
    """
    points = np.array(points)
    
    # Represent the points by their indices in points
    points_left = np.arange(len(points)) # [P]

    # Initialise an array for the sampled indices
    sample_inds = np.zeros(n_samples, dtype='int') # [S]

    # Initialise distances to inf
    dists = np.ones_like(points_left) * float('inf') # [P]

    # Select a point from points by its index, save it
    selected = 0
    sample_inds[0] = points_left[selected]

    # Delete selected 
    points_left = np.delete(points_left, selected) # [P - 1]

    # Iteratively select points for a maximum of n_samples
    for i in range(1, n_samples):
        # Find the distance to the last added point in selected
        # and all the others
        last_added = sample_inds[i-1]
        
        dist_to_last_added_point = (
            (points[last_added] - points[points_left])**2).sum(-1) # [P - i]

        # If closer, updated distances
        dists[points_left] = np.minimum(dist_to_last_added_point, 
                                        dists[points_left]) # [P - i]

        # We want to pick the one that has the largest nearest neighbour
        # distance to the sampled points
        selected = np.argmax(dists[points_left])
        sample_inds[i] = points_left[selected]

        # Update points_left
        points_left = np.delete(points_left, selected)

    return points[sample_inds]

In [68]:
sampled_points = fps(verts, 1024)

In [69]:
fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z,
                                   mode='markers')])
fig.show()

In [80]:
fig = go.Figure(data=[go.Scatter3d(
    x=sampled_points[:,0],
    y=sampled_points[:,1],
    z=sampled_points[:,2],
    mode='markers',
    marker=dict(
        size=6,
        opacity=0.8
    )
)])

# tight layout
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()

In [71]:
sampled_points.shape

(1024, 3)

In [72]:
sampled_points200 = fps(verts, 200)

In [75]:
fig = go.Figure(data=[go.Scatter3d(x=sampled_points200[:,0], y=sampled_points200[:,1], z=sampled_points200[:,2], mode='markers')])
fig.show()

In [79]:
fig = go.Figure(data=[go.Scatter3d(
    x=sampled_points200[:,0],
    y=sampled_points200[:,1],
    z=sampled_points200[:,2],
    mode='markers',
    marker=dict(
        size=6,
        opacity=0.8
    )
)])

# tight layout
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()